In [3]:
import FlowCytometryTools

In [9]:
import os

In [11]:
datafile = '/Users/kangakum/Desktop/FlowCytometry/CAP 01  2019 - AML/CAP 01 2019_Tube 6_001.fcs'

In [6]:
from FlowCytometryTools import FCMeasurement


In [12]:
sample = FCMeasurement(ID = 'Test Sample', datafile = datafile)

In [13]:
print(sample.channel_names)

('FSC-A', 'FSC-H', 'SSC-A', 'FITC-A', 'PE-A', 'PerCP-Cy5-5-A', 'PE-Cy7-A', 'APC-A', 'APC-R700-A', 'APC-H7-A', 'V450-A', 'V500-C-A', 'BV605-A', 'Time')


In [14]:
print(sample.channels)

                         $PnN    $PnR  $PnB    $PnE  $PnV  $PnG
Channel Number                                                 
1                       FSC-A  262144    32  [0, 0]   141   1.0
2                       FSC-H  262144    32  [0, 0]   141   1.0
3                       SSC-A  262144    32  [0, 0]   335   1.0
4                      FITC-A  262144    32  [0, 0]   391   1.0
5                        PE-A  262144    32  [0, 0]   458   1.0
6               PerCP-Cy5-5-A  262144    32  [0, 0]   637   1.0
7                    PE-Cy7-A  262144    32  [0, 0]   543   1.0
8                       APC-A  262144    32  [0, 0]   458   1.0
9                  APC-R700-A  262144    32  [0, 0]   585   1.0
10                   APC-H7-A  262144    32  [0, 0]   525   1.0
11                     V450-A  262144    32  [0, 0]   339   1.0
12                   V500-C-A  262144    32  [0, 0]   372   1.0
13                    BV605-A  262144    32  [0, 0]   500   1.0
14                       Time  262144   